In [1]:
import pandas as pd
import numpy as np
import rdflib
from rdflib.namespace import RDF
from rdflib import Namespace
from rdflib import URIRef, Literal

In [2]:
global g10
g10=rdflib.Graph()
g10.parse('C:/Users/SESA456791/Desktop/Ontology-Bh/Files/Outputs/Graph_Judiciary_9.nt',format='nt')
print(len(g10))

105784


In [3]:
#************Change Path to files************
international_data=pd.read_csv("C:/Users/SESA456791/Desktop/Ontology-Bh/Files/international_new.csv",header=None) 
international_data=international_data.replace(np.nan,"",regex=True)

In [4]:
international_data.columns=['Cat1','Cat2','Org1','Org2']
international_data.head(5)

,Cat1,Cat2,Org1,Org2
0,International,Embassy of India,['AFGHANISTAN'],['Embassy of India Kabul Afghanistan' 'http://...
1,International,Embassy of India,['ALGERIA'],['Embassy of India Algiers Algeria' 'http://ww...
2,International,Embassy of India,['ANGOLA'],['Embassy of India Luanda Angola' 'https://ind...
3,International,Embassy of India,['ARGENTINA'],['Embassy of India Buenos Aires Argentina' 'ht...
4,International,Embassy of India,['ARMENIA'],['Embassy of India Yerevan Armenia' 'https://e...


In [5]:
global org
global bhorg
global skos
global bhcatres
global bhorgres

org=Namespace("http://www.w3.org/ns/org#")
bhorg=Namespace("http://semanticwebindia.in/bharathi/ont/org/")
skos=Namespace("http://www.w3.org/2004/02/skos/core#")
bhcatres=Namespace("http://semanticwebindia.in/bharathi/res/cat/")
bhorgres=Namespace("http://semanticwebindia.in/bharathi/res/org/")

In [6]:
pd.set_option('display.max_colwidth', -1)#housekeeping

In [7]:
#finding n_org max
global n_org
l_lit=[]
for s,p,o in g10.triples((None, RDF.type, bhorg.Organisation)):
    str_s_uri=str(s)
    pos=str_s_uri.rfind('/')
    num=int(str_s_uri[pos+1:len(str_s_uri)])
    num_s=str(num)
    num_s=num_s[1:]
    num_i=int(num_s)
    l_lit.append(num_i) #list of n_org identifier values
#print(len(l_lit))
l_lit.sort() #sorting the list
n_org=l_lit[-1] #the current maximum value of n_org

#finding n_cat max, 
global n_cat
n_cat=0
cat_list=[]
for s,p,o in g10.triples((None, skos.prefLabel, None)):
    str_s_uri=str(s)
    pos=str_s_uri.rfind('/')
    if str_s_uri[pos+1:len(str_s_uri)]!='topOT':
        num=int(str_s_uri[pos+1:len(str_s_uri)])
        num_s=str(num)
        num_s=num_s[1:]
        num_i=int(num_s)
        cat_list.append(num_i) #list of n_org identifier values
cat_list.sort() #sorting the list
n_cat=cat_list[-1] #the current maximum value of n_org

In [8]:
print(n_cat)
print(n_org)

55
11264


In [9]:
#create fetch function for cat_id
def fetch_n_cat(cat):
    global n_cat
    global g10
    cat_id=""
    for s,p,o in g10.triples((None, skos.prefLabel,Literal(cat))):
        cat_id=s
    if cat_id=="":
        bhcatres="http://semanticwebindia.in/bharathi/res/cat/"
        n_cat+=1
        cat_id=URIRef(bhcatres+str('2')+str(n_cat))
        g10.add((cat_id,RDF.type,skos.Concept))
        g10.add((cat_id,skos.prefLabel, Literal(cat)))
        g10.add((cat_id,skos.altLabel, Literal(cat)))
        g10.add((cat_id,skos.inScheme,URIRef(bhcatres+'OrganisationType')))
    return(cat_id)

#create fetch function for org_id
def fetch_n_org(org_nam,org_url="",org_abb=""):
    global g10
    global n_org
    org_id=""
    for s,p,o in g10.triples((None, bhorg.hasName,Literal(org_nam))):
        org_id=s
    if org_id=="":
        bhorgres="http://semanticwebindia.in/bharathi/res/org/"
        n_org+=1
        org_id=URIRef(bhorgres+str('1')+str(n_org))
        g10.add((org_id, bhorg.hasName, Literal(org_nam)))
        g10.add((org_id, bhorg.hasAlternateName, Literal(org_nam)))
        if org_url!="":
            g10.add((org_id, bhorg.hasURL, Literal(org_url)))
        if org_abb!="":
            g10.add((org_id, bhorg.hasAbbreviation, Literal(org_abb)))
        g10.add((org_id,RDF.type,bhorg.Organisation))
        g10.add((org_id,org.identifier,Literal(n_org)))
        g10.add((org_id,org.hasOrganizationType,URIRef(bhcatres+str('21')))) #skos(Government) :bhcatres/21
    return(org_id)

In [10]:
#create fetch_cat_broader for broader relation
def fetch_cat_broader(cat1_id,cat2_id):
    global g10
    if not (cat2_id, skos.broader,cat1_id) in g10:
        g10.add((cat2_id, skos.broader,cat1_id))
    if not (cat1_id, skos.broaderOf,cat2_id) in g10:
        g10.add((cat1_id, skos.broaderOf,cat2_id))
        
#create fetch_org_subOrg for sub-organization relation
def fetch_org_subOrg(org1_id,org2_id):
    global g10
    if not (org1_id,org.hasSubOrganization,org2_id) in g10:
        g10.add((org1_id,org.hasSubOrganization,org2_id))
    if not (org2_id,org.subOrganizationOf,org1_id) in g10:
        g10.add((org2_id,org.subOrganizationOf,org1_id))

In [11]:
def org_scrub(org_n):
    l_scrub=[]
    val=org_n
    val=str(val)
    val=val.replace('[',"")
    val=val.replace(']',"")
    if len(val)==0:
        return(l_scrub)
    #print(val)
    pos_url=val.find('http')
    #print(pos_url)
    if pos_url!=-1:
        val_1=val[:pos_url]+'|'+val[pos_url:]
        pos_end_url=val_1[pos_url+1:].find("'")
        val_2=val_1[:pos_url+1+pos_end_url]+'|'+val_1[pos_url+1+pos_end_url:]
    else:
        sch_ch=val[0]
        pos_end=val[1:].find(sch_ch)
        #print(val[:pos_end+2])
        val_2=val[:pos_end+2]+"|"+val[pos_end+2:]
        #print(val_2)
    org_ls=val_2.split('|')
    
    for i in range(len(org_ls)):
        ele=org_ls[i]
        ele_ls=list(ele)
        clean_c=1
        while clean_c>0 and len(ele_ls)>0:
            if ele_ls[0] in ["'",'"',',',' '] and len(ele_ls)>0:
                ele_ls.pop(0)
                #print(ele_ls)
            if len(ele_ls)>0:
                if ele_ls[0] in ["'",'"',',',' ']:
                    clean_c=1
                else:
                    clean_c=0
            else:
                clean_c=0
        clean_c=1
        while clean_c>0 and len(ele_ls)>0:
            if ele_ls[-1] in ["'",'"',',',' ']:
                ele_ls.pop(-1)
            if len(ele_ls)>0:
                if ele_ls[-1] in ["'",'"',',',' ']:
                    clean_c=1
                else:
                    clean_c=0
            else:
                clean_c=0
        ele_j=''.join(ele_ls)
        l_scrub.append(ele_j)
    return(l_scrub)

In [12]:
#Iteration over legislature_data rows
for i in range(international_data.shape[0]):
    
    cat1=international_data.iloc[i,0].strip()
    cat2=international_data.iloc[i,1].strip()
    org1=international_data.iloc[i,2]
    org2=international_data.iloc[i,3]
        
    #check if cat1_id already exists, else create
    cat1_id=fetch_n_cat(cat1)
    
    #check if ((cat1 broader cat21)), ((cat21 broaderOf cat1)) exist, else create
    cat_govt=URIRef(bhcatres+'21')
    fetch_cat_broader(cat_govt,cat1_id)

    #check if cat2_id already exists, else create
    cat2_id=fetch_n_cat(cat2)
    
    #check if ((cat1 broader cat2)), ((cat2 broaderOf cat2)) exist, else create
    fetch_cat_broader(cat1_id,cat2_id)
    
    #check if org2_id already exists, else create
    org2_ls=org_scrub(org2)
    if (len(org2_ls))>0: 
        if len(org2_ls)==1:
            org_nam=org2_ls[0]
        elif len(org2_ls)==2:
            org_nam=org2_ls[0]
            if org2_ls[1].find('http')>-1:
                org_url=org2_ls[1]
            else:
                org_abb=org2_ls[1]
        else:
            org_nam=org2_ls[0]
            org_url=org2_ls[1]
            org_abb=org2_ls[2]
        org2_id=fetch_n_org(org_nam,org_url,org_abb)
    #create tuple org2, org:classificatoin, cat2_id
        g10.add((org2_id,org.classification,cat2_id))

In [13]:
len(g10)

106938

In [14]:
g10.serialize(format='nt', destination='C:/Users/SESA456791/Desktop/Ontology-Bh/Files/Outputs/Graph_International_10.nt')

In [15]:
u=Literal("Embassy of India Kabul Afghanistan")
for s,p,o in g10.triples((None,bhorg.hasName,u)):
    print(s,p,o)

http://semanticwebindia.in/bharathi/res/org/111265 http://semanticwebindia.in/bharathi/ont/org/hasName Embassy of India Kabul Afghanistan
